In [ ]:
%load_ext autoreload
%autoreload 2
# %cd /p/mmcardiac/Jerry/code/CINE-DENSE-Translation
%cd /home/jx8fh/Projects/LaMoD
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from modules.data import check_dict

In [ ]:
strain_prev_filename = '/scratch/jx8fh/2024-04-24-whole-vol-preds-strain-and-errors.npy'
strain_prev_data = np.load(strain_prev_filename, allow_pickle=True).tolist()
check_dict(strain_prev_data)

In [ ]:
# strain_transUNet_filename = '/p/mmcardiac/Jerry/code/CINE-DENSE-Translation/review/2024-04/2024-04-24-whole-vol-preds/2024-04-24-whole-vol-preds-strain-and-errors.npy'
strain_transUNet_filename = '/scratch/jx8fh/2024-06-22-whole-vol-preds-strain-and-errors-TransUNet.npy'
strain_transUNet_data = np.load(strain_transUNet_filename, allow_pickle=True).tolist()
check_dict(strain_transUNet_data)

In [ ]:
test_GT_data = np.load('/scratch/jx8fh/2024-04-16-test-data.npy', allow_pickle=True).tolist()
check_dict(test_GT_data[0])

In [ ]:
segment_names = [
    'Inferoseptal',
    'Inferior',
    'Inferolateral',
    'Anterolateral',
    'Anterior',
    'Anteroseptal'
]

In [ ]:
datum_idx = 95
datum_repeat_preds_fname = glob.glob(str(data_dir / f'LaMoD-DENSE-SkipDiffusion-{target_idx:03d}-*.npy'))[0]
    datum_repeat_preds = np.load(datum_repeat_preds_fname, allow_pickle=True).tolist()

    strainmat_GT = test_GT_data[datum_idx]['DENSE_strain_CCs6']
    strainmat_GT_max = np.max(strainmat_GT[~np.isnan(strainmat_GT)])    
    strainmat_GT_min = np.min(strainmat_GT[~np.isnan(strainmat_GT)])

    strainmat_MotionDecoder = strain_prev_data['MotionDecoder1x1']['DENSE_strain_segmental_pred'][datum_idx]['CCs6']

In [ ]:
import re

def filter_filenames_by_idx(filenames, target_idx):
    # Pattern matches: LaMoD-cine-{idx}-{id}-{location}.npy
    # where id can be numeric or UVA format, and location can be positive/negative
    pattern = r'LaMoD-DENSE-(\d+)-([A-Za-z0-9]+)-(-?\d+)\.npy'
    
    matching_files = []
    for filename in filenames:
        match = re.match(pattern, filename)
        if match and match.group(1) == str(target_idx).zfill(3):
            matching_files.append(filename)
    
    return matching_files

import glob
from pathlib import Path
# data_dir = Path('/scratch/jx8fh/2025-01-16-LaMoD-variability-split-100/DENSE_strain/')
data_dir = Path('/scratch/jx8fh/2025-01-04-LaMoD-variability/DENSE/')
all_fnames = glob.glob(str(data_dir / 'LaMoD-DENSE-*.npy'))
target_idx = 41
# matching_files = [Path(data_dir, matched_fname) for matched_fname in filter_filenames_by_idx([Path(f).name for f in all_fnames], target_idx)]
matching_files = glob.glob(str(data_dir / f'LaMoD-DENSE-SkipDiffusion-{target_idx:03d}-*.npy'))
print(matching_files)

In [ ]:
def create_image_gif(image_video, output_path, fps=10, subsample_rate=1):
    """
    Create a GIF from an image video.
    
    :param image_video: numpy array of shape (H, W, T)
    :param output_path: path to save the output GIF
    :param fps: frames per second for the output GIF
    :param subsample_rate: rate at which to subsample the images
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(image_video[::subsample_rate, ::subsample_rate, 0], cmap='gray')
    ax.axis('off')

    def animate(i):
        im.set_data(image_video[::subsample_rate, ::subsample_rate, i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=image_video.shape[2], interval=100, blit=True)
    anim.save(output_path, writer='ffmpeg', fps=fps)
    plt.close(fig)